# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [28]:
import pandas as pd

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the ULR using the `requests` library.

In [29]:
# YOUR CODE HERE
import requests
response = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")

Use Beautiful Soup to parse this string into a tree called `soup`

In [30]:
# YOUR CODE HERE
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">
```

There are many `<table>` tags on the page.

In [31]:
tables = soup.find_all("table")

#by looking at html i think its table 3 
table = tables[2]

We can use attributes like `class=` and `style=` to narrow down the list.

In [32]:
len(soup.find_all("table",
                  attrs={
                      "class": "wikitable sortable",
                      "style": "text-align:center"}
                  ))

                  #code doesn't work

                  

0

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [33]:
table = soup.find_all("table",
                  attrs={
                      "class": "wikitable sortable",
                      "style": "text-align:center"}
                  )[0]

                  #code doesn't work

IndexError: list index out of range

**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [ ]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [34]:
#turn html into actual dataframe

# initialize an empty list
rows = []

# iterate over all rows in the city table
for city in table.find_all("tr")[2:]:

    # Get all the cells (<td>) in the row.
    cells = city.find_all("td")

    # The information we need is the text between tags.

    # Find the the name of the city in cell[0]
    # which for most city is contained in the <a> tag
    city_tag = cells[0].find("a") or cells[0]
    city = city_tag.text

    # Find the office of the faculty in cell[1]
    # which for most faculty is contained in the <a> tag
    state_tag = cells[1].find("a") or cells[1]
    state = state_tag.text

    # Find the email of the faculty in cell[3]
    # which for most faculty is contained in the <a> tag
    estimate_2023_tag = cells[2].find("td") or cells[2]
    estimate_2023 = estimate_2023_tag.text

    census_2020_tag = cells[3].find("td") or cells[3]
    census_2020 = census_2020_tag.text

    change_tag = cells[4].find("td") or cells[4]
    change = change_tag.text

    land_area_miles_squared_tag = cells[5].find("td") or cells[5]
    land_area_miles_squared = land_area_miles_squared_tag.text

    land_area_kilometer_squared_tag = cells[6].find("td") or cells[6]
    land_area_kilometer_squared = land_area_kilometer_squared_tag.text

    density_miles_squared_tag = cells[7].find("td") or cells[7]
    density_miles_squared = density_miles_squared_tag.text

    density_kilometer_squared_tag = cells[8].find("td") or cells[8]
    density_kilometer_squared = density_kilometer_squared_tag.text

    location_tag = cells[9].find("small") or cells[9]
    location = location_tag.text

    # Append this data.
    rows.append({
        "city": city,
        "state": state,
        "2023 estimate": estimate_2023,
        "2023 census": census_2020,
        "Change": change,
        "land area (miles^2)": land_area_miles_squared,
        "landarea (kilometer^2)": land_area_kilometer_squared,
        "density (miles^2)": density_miles_squared,
        "density (kilometer^2)": density_kilometer_squared,
        "location": location
    })


In [35]:
pd.DataFrame(rows)

,city,state,2023 estimate,2023 census,Change,land area (miles^2),landarea (kilometer^2),density (miles^2),density (kilometer^2),location
0,New York,NY,"8,258,035\n","8,804,190\n",−6.20%\n,300.5\n,778.3\n,"29,298\n","11,312\n",40°40′N 73°56′W﻿ / ﻿40.66°N 73.94°W﻿ / 40.66; ...
1,Los Angeles,CA,"3,820,914\n","3,898,747\n",−2.00%\n,469.5\n,"1,216.0\n","8,304\n","3,206\n",34°01′N 118°25′W﻿ / ﻿34.02°N 118.41°W﻿ / 34.02...
2,Chicago,IL,"2,664,452\n","2,746,388\n",−2.98%\n,227.7\n,589.7\n,"12,061\n","4,657\n",41°50′N 87°41′W﻿ / ﻿41.84°N 87.68°W﻿ / 41.84; ...
3,Houston,TX,"2,314,157\n","2,304,580\n",+0.42%\n,640.4\n,"1,658.6\n","3,599\n","1,390\n",29°47′N 95°23′W﻿ / ﻿29.79°N 95.39°W﻿ / 29.79; ...
4,Phoenix,AZ,"1,650,070\n","1,608,139\n",+2.61%\n,518.0\n,"1,341.6\n","3,105\n","1,199\n",33°34′N 112°05′W﻿ / ﻿33.57°N 112.09°W﻿ / 33.57...
...,...,...,...,...,...,...,...,...,...,...
331,Yuma,AZ,"100,858\n","95,548\n",+5.56%\n,120.7\n,312.6\n,792\n,306\n,32°31′N 114°31′W﻿ / ﻿32.52°N 114.52°W﻿ / 32.52...
332,New Bedford,MA,"100,695\n","101,079\n",−0.38%\n,20.0\n,51.8\n,"5,054\n","1,951\n",41°40′N 70°56′W﻿ / ﻿41.66°N 70.94°W﻿ / 41.66; ...
333,Suffolk,VA,"100,659\n","94,324\n",+6.72%\n,399.2\n,"1,033.9\n",236\n,91\n,36°42′N 76°38′W﻿ / ﻿36.70°N 76.63°W﻿ / 36.70; ...
334,Hesperia,CA,"100,633\n","99,818\n",+0.82%\n,72.7\n,188.3\n,"1,373\n",530\n,34°24′N 117°19′W﻿ / ﻿34.40°N 117.32°W﻿ / 34.40...


Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Aside: Scraping an HTML table with Pandas



The Pandas command `read_html` can be used to scrape information from an HTML table on a webpage.

We can call `read_html` on the URL.

In [36]:
pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")

[                    Population tables of U.S. cities
 0  The skyline of New York City, the most populou...
 1                                             Cities
 2  Population AreaDensityEthnic identityForeign-b...
 3                                        Urban areas
 4             Populous cities and metropolitan areas
 5                                 Metropolitan areas
 6  184 combined statistical areas935 core-based s...
 7                                        Megaregions
 8  See related population listsNorth American met...
 9                                                vte,
     0                               1
 0 NaN                   State capital
 1 NaN              State largest city
 2 NaN  State capital and largest city
 3 NaN                 Federal capital,
             City  ST  ... 2020 density                               Location
             City  ST  ...        / km2                               Location
 0    New York[c]  NY  ...        11312    40°40′N 

However, this scrapes all the tables on the webpage, not just the one we want. As with Beautiful Soup, we can narrow the search by specifying the table attributes.

In [37]:
pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'wikitable sortable', "style": "text-align:center"})

#the filtering doesn't work but can use the 3 table in wiki

ValueError: No tables found

This still returns 3 tables. As we remarked above, the table that we want is the first one (see `[0]` below).

In [ ]:
df_cities2 = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")[2]
df_cities2

#the filtering doesn't work but can use the 3 table in wiki

Wait, that seemed much easier than using Beautiful Soup, and it returned a data frame, and we even got for free some formatting like removing the commas from the population! Why didn't we just use `read_html` in the first place? It's true the `read_html` works well when scraping information from an HTML *table*. Unfortunately, you often want to scrape information from a webpage that isn't conveniently stored in an HTML table, in which case `read_html` won't work. (It only searches for `<table>`, `<th>`, `<tr>`, and `<td>` tags, but there are many other HTML tags.) Though Beautiful Soup is not as simple as `read_html`, it is more flexible and thus more widely applicable.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [ ]:
pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [38]:
# YOUR CODE HERE
import requests
response = requests.get("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

In [39]:
# YOUR CODE HERE
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")



In [41]:
page = soup.find_all("div",
                  attrs={
                      "id": "courseinventorycontainer"}
                  )

len(page)

1

In [45]:
page = soup.find_all("div", {"class": "courseblock"})

In [76]:
#turn html into actual dataframe

# initialize an empty list
rows = []

# iterate over all vals in the courses
#each course is seperated by div
for courseblock in page:

    # course_title
    course_tag = courseblock.find("p", {"class": "courseblocktitle"})
    course = course_tag.get_text().strip()

    #term offered
    term_tag = courseblock.find("div", {"class": "noindent courseextendedwrap"})
    term = term_tag.get_text().strip()
    # Append this data.
    rows.append({
        "course": course,
        "term offered": term
    })


In [77]:
pd.DataFrame(rows)

,course,term offered
0,DATA 100. Data Science for All I.,"Term Typically Offered: F, W, SP2020-21 or lat..."
1,DATA 301. Introduction to Data Science.,"Term Typically Offered: F, W, SPPrerequisite: ..."
2,DATA 401. Data Science Process and Ethics.,Term Typically Offered: FPrerequisites: DATA 3...
3,DATA 402. Mathematical Foundations of Data Sci...,Term Typically Offered: FPrerequisites: CSC 46...
4,DATA 403. Data Science Projects Laboratory.,Term Typically Offered: FConcurrent: DATA 401 ...
...,...,...
69,STAT 551. Statistical Learning with R.,Term Typically Offered: FPrerequisite: STAT 30...
70,STAT 566. Graduate Consulting Practicum.,Term Typically Offered: SPPrerequisite: STAT 4...
71,STAT 570. Selected Advanced Topics.,Term Typically Offered: TBDPrerequisite: Gradu...
72,STAT 590. Graduate Seminar in Statistics.,"Term Typically Offered: F, W, SPPrerequisite: ..."


Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag